<a href="https://colab.research.google.com/github/Ram0506-eng/Ramanathan-K/blob/main/Copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision ultralytics tensorflow opencv-python pandas


In [ ]:
from google.colab import files

# Upload test_food.jpg and calories.csv
uploaded = files.upload()
label,calories_per_serving
pizza,266
banana,105
apple,52
hamburger,295
fried_egg,90
sandwich,250



In [ ]:
import cv2
import torch
import pandas as pd
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from google.colab.patches import cv2_imshow

# Config
CALORIES_CSV = "calories.csv"
YOLO_MODEL = "yolov5s"
CLASSIFIER_CONF_TH = 0.35
DETECTION_CONF_TH = 0.25
MIN_BOX_AREA = 500

# Load calorie table
cal_df = pd.read_csv(CALORIES_CSV)
if "label" not in cal_df.columns or "calories_per_serving" not in cal_df.columns:
    raise ValueError("calories.csv must have 'label' and 'calories_per_serving' columns")

# Load YOLOv5 model
yolo_model = torch.hub.load("ultralytics/yolov5", YOLO_MODEL, pretrained=True)

# Load classifier
classifier = MobileNetV2(weights="imagenet")

def classify_food(img_crop):
    img_rgb = cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (224, 224))
    arr = image.img_to_array(img_resized)
    arr = np.expand_dims(arr, axis=0)
    arr = preprocess_input(arr)
    preds = classifier.predict(arr, verbose=0)
    decoded = decode_predictions(preds, top=1)[0]
    label = decoded[0][1]
    confidence = float(decoded[0][2])
    return label, confidence

def estimate_calories_multi(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    h, w = img.shape[:2]

    results = yolo_model(img, conf=DETECTION_CONF_TH)
    detections = results.xyxy[0].cpu().numpy()

    for det in detections:
        x1, y1, x2, y2, det_conf, cls_id = det
        if det_conf < DETECTION_CONF_TH:
            continue

        x1, y1, x2, y2 = map(int, [max(0, x1), max(0, y1), min(w - 1, x2), min(h - 1, y2)])
        if (x2 - x1) * (y2 - y1) < MIN_BOX_AREA:
            continue

        crop = img[y1:y2, x1:x2]
        if crop.size == 0:
            continue

        label, clf_conf = classify_food(crop)
        if clf_conf < CLASSIFIER_CONF_TH:
            continue

        match = cal_df[cal_df['label'].str.lower() == label.lower()]
        calories = match.iloc[0]['calories_per_serving'] if not match.empty else "N/A"

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        text = f"{label} - {calories} cal ({clf_conf*100:.1f}%)"
        cv2.putText(img, text, (x1, max(y1 - 10, 20)), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 255, 255), 2, cv2.LINE_AA)

    cv2_imshow(img)  # Works in Colab

# Run function on your test image
estimate_calories_multi("test_food.jpg")
